In [1]:
from data_extraction.data_extraction import *
import pandas

In [2]:
URL_QS = 'https://en.wikipedia.org/wiki/List_of_former_European_colonies'
r = requests.get(URL_QS)
soup = BeautifulSoup(r.text, 'lxml')

In [3]:
colonizer_countries = []

# To find the name of all colonizer country of Europe, we take care about 2 things :
#  - If there is more than 2 spaces in the text -> isn't a country
#  - If there is less than 2 letters in the text -> isn't a country

for country in soup.findAll('p'):
    nb_space = sum(c.isspace() for c in country.text)
    
    if country.text not in colonizer_countries and nb_space < 3 and len(country.text) > 2:
        colonizer_countries.append(country.text)

# List Cleaning
colonizer_countries = [country.replace('\n', '') for country in colonizer_countries]

In [4]:
regions = []

for region in soup.find("ul").findAll('li'):
    regions.append(region.text[2:])

del regions[-1], regions[-1] 

In [5]:
# Dataframe creation
colonization_df = pd.DataFrame(columns=['Colonized Country', 'ID', 'Day', 'Month', 'Year', 'Colonizer Country', "URL"])

# Check all colonizer
for colonizer_country in colonizer_countries:
    #colonizer_country = "Portugal"
    print(colonizer_country, "...")

    # Trouve tout les emplacements où on fait mention du pays "colonizer"
    colonizer_index = soup.find_all("p", text=colonizer_country) 

    for i in range (len(colonizer_index)):
        colonized_countries = colonizer_index[i].next_sibling.next_sibling('a')

        for colonized_country in colonized_countries:
             if colonized_country.text not in regions and len(colonized_country.attrs) < 3:
                check_country(colonization_df, colonized_country.text, 
                              colonizer_country, colonizer_countries, 
                              colonized_country['href'], 1)

colonization_df.head()

Denmark ...
France ...
Netherlands ...
Norway ...
Portugal ...
Russia ...
Spain ...
Sweden ...
Brandenburg ...
Britain ...
Courland ...
Denmark-Norway ...
Knights of Malta ...
Belgium ...
Germany ...
Italy ...
Austria ...
United Kingdom ...
Germany ...
Greece ...


,Colonized Country,ID,Day,Month,Year,Colonizer Country,URL
1,Greenland,GL,NaN,NaN,NaN,Denmark,/wiki/Greenland
2,Iceland,IS,NaN,NaN,NaN,Denmark,/wiki/Iceland
3,Canada,CA,1,07,1867,United Kingdom,/wiki/Canada
4,United States,US,4,07,1776,France,/wiki/United_States
5,Haiti,HT,1,01,1804,France,/wiki/Haiti


In [6]:
# Dataset cleaning
colonization_df_cleaned = colonization_df.replace("Britain", "United Kingdom")

# Add missing countries
colonization_df_cleaned.loc[len(colonization_df_cleaned)+1] = ['Algeria', "DZ", "3", "07", "1945", "France", "/wiki/Algeria"]
colonization_df_cleaned.loc[len(colonization_df_cleaned)+1] = ['Syria', "SY", "24", "10", "1945", "France", "/wiki/Syria"]

# Dataframe Saving
colonization_df_cleaned.to_csv("datasets/colonies_wikipedia.csv")

colonization_df_cleaned

,Colonized Country,ID,Day,Month,Year,Colonizer Country,URL
1,Greenland,GL,NaN,NaN,NaN,Denmark,/wiki/Greenland
2,Iceland,IS,NaN,NaN,NaN,Denmark,/wiki/Iceland
3,Canada,CA,1,07,1867,United Kingdom,/wiki/Canada
4,United States,US,4,07,1776,France,/wiki/United_States
5,Haiti,HT,1,01,1804,France,/wiki/Haiti
6,Benin,BJ,1,08,1960,France,/wiki/Benin
7,Cameroon,CM,1,01,1960,France,/wiki/Cameroon
8,Chad,TD,11,08,1960,France,/wiki/Chad
9,Republic of the Congo,CG,15,08,1960,France,/wiki/Republic_of_the_Congo
10,Central African Republic,CF,13,08,1960,France,/wiki/Central_African_Republic
